# 批量归一化

批量归一化层

对全连接层作批量归一化，可以按以下步骤实现：

1. 按特征维将输入数据拆分为大小为 $n$ 的数据块，其中 $n$ 是批量大小。
2. 对每个数据块，计算其均值和方差，再做批量归一化。
3. 将数据块重新组合。

对卷积层作批量归一化，可以按以下步骤实现：

1. 对输入的通道维拆分为大小为 $n$ 的数据块，其中 $n$ 是通道数。
2. 对每个数据块，计算其均值和方差，再做批量归一化。
3. 将数据块重新组合。

预测时的批量归一化

* 在训练时，批量归一化层在每次迭代中计算当前小批量数据的均值和方差，并使用这些均值和方差对输入数据做批量归一化。在预测时，我们通常使用训练时最后一个小批量数据的均值和方差，或者通过移动平均对这些均值和方差进行估计。

从零实现

In [1]:
import d2lzh as d2l
from mxnet import autograd, contrib, gluon, init, nd
from mxnet.gluon import nn

def batch_norm(X, gamma, beta, moving_mean, moving_var, eps,momentum):
    # 通过autograd标记需要计算梯度
    if not autograd.is_training():
        # 如果是在预测阶段，直接使用移动平均所得的均值和方差
        X_hat = (X - moving_mean) / nd.sqrt(moving_var + eps)
    else:
        # 如果是在训练阶段，使用当前的均值和方差
        assert len(X.shape) in (2, 4)
        if len(X.shape) == 2:
            # 对于全连接层，计算特征维上的均值和方差
            mean = X.mean(axis=0)
            var = ((X - mean)**2).mean(axis=0)
        else:
            # 对于卷积层，计算通道维上的均值和方差
            mean = X.mean(axis=(0, 2, 3), keepdims=True)
            var = ((X - mean)**2).mean(axis=(0, 2, 3), keepdims=True)
        # 训练阶段，用当前的均值和方差做批量归一化
        X_hat = (X - mean) / nd.sqrt(var + eps)
        # 更新移动平均的均值和方差
        moving_mean = (1 - momentum) * moving_mean + momentum * mean
        moving_var = (1 - momentum) * moving_var + momentum * var
    # 缩放和平移
    Y = gamma * X_hat + beta
    return Y, moving_mean, moving_var

In [2]:
class BatchNorm(nn.Block):
    def __init__(self, num_features, num_dims, **kwargs):
        super(BatchNorm, self).__init__(**kwargs)
        # num_dims=2表示全连接层，num_dims=4表示卷积层
        if num_dims == 2:
            shape = (1, num_features)
        else:
            shape = (1, num_features, 1, 1)
        # gamma初始化为1，beta初始化为0
        self.gamma = self.params.get('gamma', shape=shape, init=init.One())
        self.beta = self.params.get('beta', shape=shape, init=init.Zero())
        self.moving_mean = nd.zeros(shape)
        self.moving_var = nd.zeros(shape)

    def forward(self, X):
        # 在训练模式下更新移动平均的均值和方差
        if self.moving_mean.context != X.context:
            self.moving_mean = self.moving_mean.as_in_context(X.context)
            self.moving_var = self.moving_var.as_in_context(X.context)
        Y, self.moving_mean, self.moving_var = batch_norm(
            X, self.gamma.data(), self.beta.data(), self.moving_mean,
            self.moving_var, eps=1e-5, momentum=0.9)
        return Y

使用批量归一化层的LeNet

In [3]:
net=nn.Sequential()
net.add(nn.Conv2D(6, kernel_size=5),
       BatchNorm(6, num_dims=4),
       nn.Activation('sigmoid'),
       nn.MaxPool2D(pool_size=2, strides=2),
       nn.Conv2D(16, kernel_size=5),
       BatchNorm(16, num_dims=4),
       nn.Activation('sigmoid'),
       nn.MaxPool2D(pool_size=2, strides=2),
       nn.Flatten(),
       nn.Dense(120),
       BatchNorm(120, num_dims=2),
       nn.Activation('sigmoid'),
       nn.Dense(84),
       BatchNorm(84, num_dims=2),
       nn.Activation('sigmoid'),
       nn.Dense(10))

In [6]:
lr,num_epochs,batch_size=0.9,5,256
ctx=d2l.try_gpu()
net.initialize(init.Normal(sigma=0.01), ctx=ctx, force_reinit=True)
trainer=gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr})
train_iter,test_iter=d2l.load_data_fashion_mnist(batch_size)
d2l.train_ch5(net, train_iter, test_iter, batch_size, trainer, ctx, num_epochs)

training on cpu(0)
epoch 1, loss 0.8537, train acc 0.681, test acc 0.808, time 57.2 sec
epoch 2, loss 0.4769, train acc 0.828, test acc 0.842, time 53.3 sec
epoch 3, loss 0.3950, train acc 0.857, test acc 0.871, time 50.4 sec
epoch 4, loss 0.3598, train acc 0.871, test acc 0.880, time 48.5 sec
epoch 5, loss 0.3388, train acc 0.877, test acc 0.880, time 48.1 sec
